# MP5: Weighted Finite State Transducers

In this MP, you will train and test a nested set of two WFSTs: a language model, and a lexicon.  The file you're looking at right now (mp5overview.ipynb) is a debugging tool.  The file you actually need to complete is mp5.py.  The unit tests are provided in run_tests.py and tests/test_visible.py.  All of these are available as part of the code package, https://courses.engr.illinois.edu/ece417/fa2020/ece417_20fall_mp5.zip.

In [14]:
import numpy  as np
import matplotlib.figure
import matplotlib.pyplot as plt
import codecs
%matplotlib inline

In [15]:
import mp5
import importlib
importlib.reload(mp5)

<module 'mp5' from 'C:\\Users\\66403\\UIUC_Courses\\ECE417\\ece417_20fall_mp5\\mp5.py'>

## How to debug

In order to reduce the length of this overview, every block below has two options: you can either show your own results, or you can show the distributed solutions.  In order to decide which one you want to see, you should just comment out the other one.

In [16]:
import json
with open('solutions.json') as f:
    solutions = json.load(f)

## Browsing the Data

This MP will simulate training an automatic speech recognizer from an untranscribed input utterance.  
* The audio file is available in the data folder, but we won't load it. 
* Instead, we'll assume that a phoneme transcription system has already converted it into a sequence of phonemes, and your goal is to figure out what words were spoken.

Here are the phonemes that were "recognized" in the input audio file:

In [17]:

with codecs.open('data/transcript.txt',encoding='utf-8') as f:
    S = f.read().strip().split()
print(S)

['h', 'ˈu', 'z', 'w', 'ˈʊ', 'd', 'z', 'ð', 'ˈi', 'z', 'ˈɑ', 'ɹ', 'ˈɑɪ', 'ɵ', 'ˈɪ', 'ŋ', 'k', 'ˈɑɪ', 'n', 'ˈoʊ', 'h', 'ɪ', 'z', 'h', 'ˈaʊ', 's', 'ɪ', 'z', 'ˈɪ', 'n', 'ð', 'ə', 'v', 'ˈɪ', 'l', 'ɪ', 'dʒ', 'ð', 'ˈoʊ', 'h', 'ˈi', 'w', 'ɪ', 'l', 'n', 'ˈɑ', 't', 's', 'ˈi', 'm', 'ˈi', 's', 't', 'ˈɑ', 'p', 'ɪ', 'ŋ', 'h', 'ˈɪ', 'ɹ', 't', 'ˈoʊ', 'w', 'ˈɑ', 'tʃ', 'h', 'ɪ', 'z', 'w', 'ˈʊ', 'd', 'z', 'f', 'ˈɪ', 'l', 'ˈʌ', 'p', 'w', 'ɪ', 'ð', 's', 'n', 'ˈoʊ', 'm', 'ˈi', 'l', 'ˈɪ', 't', 'l̩', 'h', 'ˈɔ', 'ɹ', 's', 'm', 'ə', 's', 't', 'ɵ', 'ˈɪ', 'ŋ', 'k', 'ɪ', 't', 'k', 'w', 'ˈɪ', 'ɹ', 't', 'ˈoʊ', 's', 't', 'ˈɑ', 'p', 'w', 'ɪ', 'ð', 'ˈaʊ', 't', 'ə', 'f', 'ˈɑ', 'ɹ', 'm', 'h', 'ˈaʊ', 's', 'n', 'ˌi', 'ɹ', 'b', 'i', 't', 'w', 'ˈi', 'n', 'ð', 'ə', 'w', 'ˈʊ', 'd', 'z', 'ə', 'n', 'd', 'f', 'ɹ', 'ˈoʊ', 'z', 'n̩', 'l', 'ˈei', 'k', 'ð', 'ə', 'd', 'ˈɑ', 'ɹ', 'k', 'ə', 's', 't', 'ˈi', 'v', 'n', 'ɪ', 'ŋ', 'ə', 'v', 'ð', 'ə', 'j', 'ˌi', 'ɹ', 'h', 'ˈi', 'g', 'ˈɪ', 'v', 'z', 'h', 'ɪ', 'z', 'h', 'ˈɑ', 'ɹ', 'n', 'ɪ', 's'

This time, you'll need to write your own code to read in the lexicon, to train a language model, and to read in the transcript as a WFST. Here's some code to load the lexicon.  Notice that, for most words, there is more than one possible pronunciation, e.g., because people sometimes pronounce things casually in  a reduced fashion:

In [18]:
with codecs.open('data/lexicon.txt', encoding='utf-8') as f:
    for line in f:
        ph = line.strip().split()
        #print(ph[0], ph[1:])
        print('Word %s is pronounced as %s'%(ph[0],':'.join(ph[1:])))

Word a is pronounced as ə
Word a is pronounced as ˌei
Word about is pronounced as b:ˌaʊ:t
Word about is pronounced as ə:b:ˈaʊ:t
Word aleksandrovich is pronounced as ˌɑ:l:ɛ:k:s:ˈɑ:n:d:ɹ:ɑ:v:ɪ:tʃ
Word alexander is pronounced as ˌæ:l:ɪ:g:z:ˈæ:n:d:ɚ
Word alexander is pronounced as ˌæ:l:ɪ:g:z:ˈæ:n:d:ə:ɹ
Word all is pronounced as ɑ:l
Word all is pronounced as ˈɔ:l
Word and is pronounced as ə:n:d
Word and is pronounced as ˈæ:n:d
Word are is pronounced as ˈɑ:ɹ
Word as is pronounced as ˈæ:z
Word as is pronounced as ˈɛ:z
Word ask is pronounced as ˈæ:s:k
Word at is pronounced as ə:t
Word at is pronounced as ˈæ:t
Word be is pronounced as b:i
Word be is pronounced as b:ˈei
Word be is pronounced as b:ˈi
Word before is pronounced as b:ɪ:f:ˈoʊ:ɹ
Word before is pronounced as b:ɪ:f:ˈɔ:ɹ
Word before is pronounced as b:ˌi:f:ˈɔ:ɹ
Word bells is pronounced as b:ˈɛ:l:z
Word between is pronounced as b:i:t:w:ˈi:n
Word between is pronounced as b:ɪ:t:w:ˈi:n
Word book is pronounced as b:ˈʊ:k
Word broke is pronounc

You will also use Laplace smoothing to train a unigram language model.  Let's look at the language model training texts:

In [19]:
text = []
with open('data/languagemodeltexts.txt') as f:
    for line in f:
        text.append(line.strip())
print(' '.join(text))

when telling of nicholas the second our temptation is to start at its dramatic end july nineteen eighteen massacre of him his entire family his household help and personal physician by which a triumphant communist movement introduced its rule  but there are more instructive stories about nicholas including his reaction to learning in november eighteen ninety four that his father alexander had died and that he nicholas was now emperor of all russia  as told by simon sebag montenori in his book twenty six year old nicholai aleksandrovich romanov broke down in tears and ran to his sister  what is going to happen to me to my family and to russia he cried on her shoulder i never wanted to be czar 


## Read the data as three WFSTs

The first thing you need to do is to read in the lexicon and the transcript in the form of WFSTs, and to train a WFST to represent the language model.  Remember that an WFST is composed of six things:
1. A set of input labels.  
2. A set of output labels.  
3. A set of states.
4. A specification of the initial state.
5. A specification of the final states.
6. A list of transitions.

Most of this MP will assume that most of these things are trivial:
1. Input labels: we'll assume that any string is possible.  
2. Output labels: any string is possible.  
3. States: non-negative integers, numbered sequentially from zero.
4. Initial state: we'll always assume 0 is the initial state.
5. Final states: this needs to be specified for each WFST.
6. Transitions: OK, this is where the real work will occur.

We will assume that "creating a WFST" is synonymous with "creating a list of 
transitions."  Each transition, $t$, needs to be a python tuple, containing the following five elements, in order:
0. t[0]=$p[t]$: The preceding state (a non-negative integer)
1. t[1]=$i[t]$: The input label (a string, or '' for epsilon)
2. t[2]=$o[t]$: The output label (a string, or '' for epsilon)
3. t[3]=$w[t]$: The weight (surprisal: a real number, or np.inf)
4. t[4]=$n[t]$: The next state (a non-negative integer)

First, let's start with the easiest WFST: the one representing the input transcription.  The number of transitions should exactly equal $N$, the number of phoneme symbols in the file data/transcript.txt.  These transitions should go through a sequence of $N+1$ states, starting with $p[0]=0$, and ending with $n[N]=N$.  The input and output labels are the same ($i[t]=o[t]$), and the weights are all zero ($w[t]=0$).  So it should look like
0. (0,'h','h',0,1)
1. (1,'u','u',0,2)
...

Obviously, the initial state is $q=0$, and the final state is $q=N$.  We won't write that down anywhere, we just need to remember it.

In [20]:
importlib.reload(mp5)
T, Tfinal =mp5.todo_transcript2wfst('data/transcript.txt')
#T = solutions['T']
#Tfinal = solutions['Tfinal']
print(Tfinal)
print(T)

[342]
[(0, 'h', 'h', 0, 1), (1, 'ˈu', 'ˈu', 0, 2), (2, 'z', 'z', 0, 3), (3, 'w', 'w', 0, 4), (4, 'ˈʊ', 'ˈʊ', 0, 5), (5, 'd', 'd', 0, 6), (6, 'z', 'z', 0, 7), (7, 'ð', 'ð', 0, 8), (8, 'ˈi', 'ˈi', 0, 9), (9, 'z', 'z', 0, 10), (10, 'ˈɑ', 'ˈɑ', 0, 11), (11, 'ɹ', 'ɹ', 0, 12), (12, 'ˈɑɪ', 'ˈɑɪ', 0, 13), (13, 'ɵ', 'ɵ', 0, 14), (14, 'ˈɪ', 'ˈɪ', 0, 15), (15, 'ŋ', 'ŋ', 0, 16), (16, 'k', 'k', 0, 17), (17, 'ˈɑɪ', 'ˈɑɪ', 0, 18), (18, 'n', 'n', 0, 19), (19, 'ˈoʊ', 'ˈoʊ', 0, 20), (20, 'h', 'h', 0, 21), (21, 'ɪ', 'ɪ', 0, 22), (22, 'z', 'z', 0, 23), (23, 'h', 'h', 0, 24), (24, 'ˈaʊ', 'ˈaʊ', 0, 25), (25, 's', 's', 0, 26), (26, 'ɪ', 'ɪ', 0, 27), (27, 'z', 'z', 0, 28), (28, 'ˈɪ', 'ˈɪ', 0, 29), (29, 'n', 'n', 0, 30), (30, 'ð', 'ð', 0, 31), (31, 'ə', 'ə', 0, 32), (32, 'v', 'v', 0, 33), (33, 'ˈɪ', 'ˈɪ', 0, 34), (34, 'l', 'l', 0, 35), (35, 'ɪ', 'ɪ', 0, 36), (36, 'dʒ', 'dʒ', 0, 37), (37, 'ð', 'ð', 0, 38), (38, 'ˈoʊ', 'ˈoʊ', 0, 39), (39, 'h', 'h', 0, 40), (40, 'ˈi', 'ˈi', 0, 41), (41, 'w', 'w', 0, 42), (42, 'ɪ'

Now let's read the lexicon.  Assuming there are no homophones (no pairs of words that sound the same), we can construct a deterministic WFST as follows:
1. The initial state is $i=0$.
2. A word containing $N$ phones is expressed as a sequence of $N+1$ transitions. 
3. The first transition in each word starts from state 0.
4. The first $N$ transitions in each word have the phone as input label, and epsilon (the empty string, '') as  output label.
5. The $N+1$st transition in each word has epsilon ('') as input string, and the word as the output label, and ends in state 0.
6. Transitions are shared, between words, whenever possible.  Two words that have the first $M$ phonemes in common will share the first $M$ transitions, and diverge on the $(M+1)$st transition.
7. For now, we'll set the edge weights all to 0.  We will change that later, during Baum-Welch re-estimation.
8. In order to make it possible for the autograder to recognize that you've done this correctly, please construct states in the order that they are required.  Each new state should be one larger than the most recently constructed state.

So if the first two entries in the lexicon were "a ə", "a ˌei" and "about ə b ˈaʊ t", then the first several transitions in the WFST would be:
* (0,"ə","",0,1)
* (1,"","a",0,0)
* (0,"ˌei","",0,2)
* (2,"","a",0,0)
* (1,"b","",0,3)
* (3,"ˈaʊ","",0,4)
* (4,"t","",0,5)
* (5,"","about",0,0)

Here's what that looks like.  Actually the third word in the lexicon is "about b ˈaʊ t" not "about ə b ˈaʊ t", so the result is a little different than the one listed above:

In [21]:
importlib.reload(mp5)
#L, Lfinal = mp5.todo_lexicon2wfst('data/lexicon.txt')
L = solutions['L']
Lfinal = solutions['Lfinal']
print(Lfinal)
print(L)

[0]
[[0, 'ə', '', 0, 1], [1, '', 'a', 0, 0], [0, 'ˌei', '', 0, 2], [2, '', 'a', 0, 0], [0, 'b', '', 0, 3], [3, 'ˌaʊ', '', 0, 4], [4, 't', '', 0, 5], [5, '', 'about', 0, 0], [1, 'b', '', 0, 6], [6, 'ˈaʊ', '', 0, 7], [7, 't', '', 0, 8], [8, '', 'about', 0, 0], [0, 'ˌɑ', '', 0, 9], [9, 'l', '', 0, 10], [10, 'ɛ', '', 0, 11], [11, 'k', '', 0, 12], [12, 's', '', 0, 13], [13, 'ˈɑ', '', 0, 14], [14, 'n', '', 0, 15], [15, 'd', '', 0, 16], [16, 'ɹ', '', 0, 17], [17, 'ɑ', '', 0, 18], [18, 'v', '', 0, 19], [19, 'ɪ', '', 0, 20], [20, 'tʃ', '', 0, 21], [21, '', 'aleksandrovich', 0, 0], [0, 'ˌæ', '', 0, 22], [22, 'l', '', 0, 23], [23, 'ɪ', '', 0, 24], [24, 'g', '', 0, 25], [25, 'z', '', 0, 26], [26, 'ˈæ', '', 0, 27], [27, 'n', '', 0, 28], [28, 'd', '', 0, 29], [29, 'ɚ', '', 0, 30], [30, '', 'alexander', 0, 0], [29, 'ə', '', 0, 31], [31, 'ɹ', '', 0, 32], [32, '', 'alexander', 0, 0], [0, 'ɑ', '', 0, 33], [33, 'l', '', 0, 34], [34, '', 'all', 0, 0], [0, 'ˈɔ', '', 0, 35], [35, 'l', '', 0, 36], [36, '', '

Finally, let's create the language model.  We will use a unigram language model, meaning that the probability of every word is independent of the words around it, e.g., the word sequence $\vec{o}=[o_1,o_2,o_3]$ is modeled as

$$p(\vec{o}) = \prod_{k=1}^3 p(o_k)$$

This can be modeled as a WFST with just one state, state 0.  Each transition, $t$, goes from $p[t]=0$ to $n[t]=0$.  The input label and output label are the same: they both equal the word.  The weight on each edge is the negative log probability of the word.  So for example, the first several edges would be
* (0,"a","a",$-\ln p($a$)$,0)
* (0,"about","about",$-\ln p($about$)$,0)
* (0,"alexandrovich","alexandrovich",$-\ln p($alexandrovich$)$,0)

The probabilities should be estimated from the file 'data/languagemodeltexts.txt', by counting the number of occurrences of each word, and Laplace-smoothing with a smoothing factor of $1$.  So if $C(w)$ is the number of times word $w$ occurs in languagemodeltexts.txt, then

$$p(w) = \frac{1+C(w)}{\sum_{v\in V} (1+C(v))}$$

where $V$ is the set of all distinct words (all words with different orthographic representations) in the lexicon.

In order to make sure that the autograder can grade your  code, please make sure that  the edges occur in the same sequence as the distinct words in the lexicon.

In [22]:
importlib.reload(mp5)
#G, Gfinal = mp5.todo_unigram('data/languagemodeltexts.txt',L)
G = solutions['G']
Gfinal = solutions['Gfinal']
print(Gfinal)
print(G)

[0]
[[0, 'a', 'a', 4.923623917106626, 0], [0, 'about', 'about', 4.923623917106626, 0], [0, 'aleksandrovich', 'aleksandrovich', 4.923623917106626, 0], [0, 'alexander', 'alexander', 4.923623917106626, 0], [0, 'all', 'all', 4.923623917106626, 0], [0, 'and', 'and', 4.007333185232471, 0], [0, 'are', 'are', 4.923623917106626, 0], [0, 'as', 'as', 4.923623917106626, 0], [0, 'ask', 'ask', 5.616771097666572, 0], [0, 'at', 'at', 4.923623917106626, 0], [0, 'be', 'be', 4.923623917106626, 0], [0, 'before', 'before', 5.616771097666572, 0], [0, 'bells', 'bells', 5.616771097666572, 0], [0, 'between', 'between', 5.616771097666572, 0], [0, 'book', 'book', 4.923623917106626, 0], [0, 'broke', 'broke', 4.923623917106626, 0], [0, 'but', 'but', 4.923623917106626, 0], [0, 'by', 'by', 4.518158808998462, 0], [0, 'communist', 'communist', 4.923623917106626, 0], [0, 'cried', 'cried', 4.923623917106626, 0], [0, 'czar', 'czar', 4.923623917106626, 0], [0, 'dark', 'dark', 5.616771097666572, 0], [0, 'darkest', 'darkest

## Composing and Searching the WFSTs

In order to figure out what words exist in the transcript, we will do the following things:
1. Compose $L$ with $G$ to create $LG=L\circ G$.  If $G$ was constructed from relevant texts, this would have the effect of telling the recognizer which words are most probable.  In our case, $G$ is made from irrelevant texts, so it will have the opposite effect.
2. Compose $T$ with $LG$ to create $TLG=T\circ LG$.  This creates a WFST that contains all and only the paths that explain the transcription (if any!)
3. Use the bestpath algorithm to find the most likely path.

The first thing we need to do is compose $L$ and $G$.  You should write todo_fstcompose.  With arguments $C=A\circ B$ your  code should implement this algorithm:
1. The set of states in $C$ is the cross product of the states in $A$, and those in $B$.  You can use  the provided function, 'cross_product'.
2. For each state $q_A\in A$, for each transition $t_B\in B$ that has an epsilon input ($i[t_B]=\epsilon$), create a transition $t_C$ that doesn't change $q_A$.
3. For each state $q_B\in B$, for each transition $t_A\in A$ with epsilon output ($o[t_A]=\epsilon$), create a transition $t_C$ that doesn't change $q_B$. 
4. For each pair of transitions $t_A$ and $t_B$ s.t. $o[t_A]=i[t_B]\ne\epsilon$, create a transition $t_C$ that changes both $q_A$ and $q_B$.
5. The set of final states for $C$ is the set of tuples $(q_A,q_B)$ for which both $q_A$ and $q_B$ are final.



In [23]:
importlib.reload(mp5)
LG, LGfinal = mp5.todo_fstcompose(L,Lfinal,G,Gfinal)
LG1 = solutions['LG']
#LGfinal = solutions['LGfinal']
#print(LGfinal)
print(len(LG))
print(len(LG1))

print(LG)
print("")
print("")
print("")
print("*************************************************************************************************************")
print("*************************************************************************************************************")
print("*************************************************************************************************************")
print("")
print("")
print("")
print(LG1)
#set(LG1) 

857
860
[(0, 'ə', '', 0, 1), (0, 'ˌei', '', 0, 2), (0, 'b', '', 0, 3), (0, 'ˌɑ', '', 0, 9), (0, 'ˌæ', '', 0, 22), (0, 'ɑ', '', 0, 33), (0, 'ˈɔ', '', 0, 35), (0, 'ˈæ', '', 0, 39), (0, 'ˈɑ', '', 0, 42), (0, 'ˈɛ', '', 0, 45), (0, 'k', '', 0, 85), (0, 'z', '', 0, 100), (0, 'd', '', 0, 103), (0, 'ˈi', '', 0, 124), (0, 'ˈei', '', 0, 127), (0, 'ɛ', '', 0, 141), (0, 'ɪ', '', 0, 146), (0, 'ˌɛ', '', 0, 151), (0, 'f', '', 0, 160), (0, 'g', '', 0, 188), (0, 'h', '', 0, 196), (0, 'ˈɑɪ', '', 0, 232), (0, 'ˈɪ', '', 0, 234), (0, 'ˌɪ', '', 0, 254), (0, 'dʒ', '', 0, 271), (0, 'n', '', 0, 283), (0, 'l', '', 0, 285), (0, 'm', '', 0, 305), (0, 'ˈoʊ', '', 0, 374), (0, 'ˈʌ', '', 0, 382), (0, 'ˈaʊ', '', 0, 387), (0, 'p', '', 0, 390), (0, 'ɹ', '', 0, 422), (0, 's', '', 0, 444), (0, 'ʃ', '', 0, 450), (0, 't', '', 0, 501), (0, 'ð', '', 0, 522), (0, 'ɵ', '', 0, 532), (0, 'v', '', 0, 556), (0, 'w', '', 0, 561), (0, 'j', '', 0, 612), (1, '', 'a', 4.923623917106626, 0), (1, 'b', '', 0, 6), (1, 'n', '', 0, 37), (1, '

Now let's compose $TLG=T\circ LG$, in order to get the set of all valid full-word paths through the transcription.  We know that the only possible final state of $LG$ is state $0$.  The only possible final state of $T$ is its largest integer, equal to the number of transitions.

In [24]:
importlib.reload(mp5)
TLG, TLGfinal = mp5.todo_fstcompose(T,Tfinal,LG,LGfinal)
TLG1 = solutions['TLG']
TLGfinal1 = solutions['TLGfinal']

print(len(TLG))
print(len(TLG1))

print(TLGfinal)
print(TLGfinal1)
print(TLG[:45])
print("*********************************")
print(TLG1[:45])

85042
86299
[215460]
[215460]
[(0, 'h', '', 0, 826), (1, '', 'a', 4.923623917106626, 0), (2, '', 'a', 4.923623917106626, 0), (5, '', 'about', 4.923623917106626, 0), (8, '', 'about', 4.923623917106626, 0), (21, '', 'aleksandrovich', 4.923623917106626, 0), (30, '', 'alexander', 4.923623917106626, 0), (32, '', 'alexander', 4.923623917106626, 0), (34, '', 'all', 4.923623917106626, 0), (36, '', 'all', 4.923623917106626, 0), (38, '', 'and', 4.007333185232471, 0), (41, '', 'and', 4.007333185232471, 0), (43, '', 'are', 4.923623917106626, 0), (44, '', 'as', 4.923623917106626, 0), (46, '', 'as', 4.923623917106626, 0), (48, '', 'ask', 5.616771097666572, 0), (49, '', 'at', 4.923623917106626, 0), (50, '', 'at', 4.923623917106626, 0), (51, '', 'be', 4.923623917106626, 0), (52, '', 'be', 4.923623917106626, 0), (53, '', 'be', 4.923623917106626, 0), (57, '', 'before', 5.616771097666572, 0), (59, '', 'before', 5.616771097666572, 0), (63, '', 'before', 5.616771097666572, 0), (66, '', 'bells', 5.616771097

Obviously, TLG contains a _lot_ of useless transitions!  Specifically, it contains a lot of transitions that are disconnected from the graph --- no way to get to them, and/or nowhere to go from them.   Since TLG contains no cycles (can you see why?), we can eliminate those useless transitions by topologically sorting the graph, using a topological sort function that only keeps transitions that can be reached from the initial state.


In [25]:
importlib.reload(mp5)
#TLG_sorted, TLGfinal_sorted = mp5.todo_sort_topologically(TLG,TLGfinal)
TLG_sorted = solutions['TLG_sorted']
TLGfinal_sorted = solutions['TLGfinal_sorted']
print(TLG_sorted[:45])

[[0, 'h', '', 0, 1], [1, 'ˈu', '', 0, 2], [2, 'z', '', 0, 3], [3, '', 'whose', 5.616771097666572, 4], [4, 'w', '', 0, 5], [5, 'ˈʊ', '', 0, 6], [6, 'd', '', 0, 7], [7, 'z', '', 0, 8], [8, '', 'woods', 5.616771097666572, 9], [9, 'ð', '', 0, 10], [10, 'ˈi', '', 0, 11], [11, '', 'the', 4.923623917106626, 12], [11, 'z', '', 0, 13], [12, 'z', '', 0, 14], [13, '', 'these', 5.616771097666572, 15], [14, 'ˈɑ', '', 0, 16], [15, 'ˈɑ', '', 0, 17], [16, 'ɹ', '', 0, 18], [17, 'ɹ', '', 0, 19], [18, '', 'czar', 4.923623917106626, 20], [19, '', 'are', 4.923623917106626, 20], [19, '', 'our', 4.923623917106626, 20], [20, 'ˈɑɪ', '', 0, 21], [21, '', 'i', 4.923623917106626, 22], [22, 'ɵ', '', 0, 23], [23, 'ˈɪ', '', 0, 24], [24, 'ŋ', '', 0, 25], [25, 'k', '', 0, 26], [26, '', 'think', 5.616771097666572, 27], [27, 'ˈɑɪ', '', 0, 28], [28, '', 'i', 4.923623917106626, 29], [29, 'n', '', 0, 30], [30, 'ˈoʊ', '', 0, 31], [31, '', 'know', 5.616771097666572, 32], [32, 'h', '', 0, 33], [33, 'ɪ', '', 0, 34], [34, 'z', 

One reason to topologically sort the graph is that it makes the bestpath algorithm, the forward algorithm, and the backward algorithm all much more efficient.

In [30]:
importlib.reload(mp5)
delta, psi, bestpath = mp5.todo_fstbestpath(TLG_sorted,TLGfinal_sorted)
bestpath1 = solutions['bestpath']
print([ t[2] for t in bestpath1 if t[2]!='' ])

print([ t[2] for t in bestpath if t[2]!='' ])

['whose', 'woods', 'the', 'czar', 'i', 'think', 'i', 'know', 'his', 'house', 'is', 'in', 'the', 'village', 'though', 'he', 'will', 'not', 'see', 'me', 'stopping', 'here', 'to', 'watch', 'his', 'woods', 'fill', 'up', 'with', 'snow', 'me', 'little', 'horse', 'must', 'think', 'it', 'queer', 'to', 'stop', 'without', 'a', 'farm', 'house', 'near', 'between', 'the', 'woods', 'and', 'frozen', 'lake', 'the', 'darkest', 'evening', 'of', 'the', 'year', 'he', 'gives', 'his', 'harness', 'bells', 'a', 'shake', 'to', 'ask', 'if', 'there', 'is', 'some', 'mistake', 'the', 'only', 'other', 'sounds', 'the', 'sweep', 'of', 'easy', 'wind', 'and', 'downy', 'flake', 'the', 'woods', 'our', 'lovely', 'dark', 'and', 'deep', 'but', 'i', 'have', 'promises', 'to', 'keep', 'and', 'miles', 'to', 'go', 'before', 'i', 'sleep', 'and', 'miles', 'to', 'go', 'before', 'i', 'sleep']
['whose', 'woods', 'the', 'czar', 'i', 'think', 'i', 'know', 'his', 'house', 'is', 'in', 'the', 'village', 'though', 'he', 'will', 'not', 'see

## Re-estimation

Re-estimation of an FST is pretty similar to an HMM.  We run the forward algorithm, then the backward algorithm, then compute xi=alpha otimes beta, and set each transition's weight equal to $w[t]=-\ln(\exp\xi_u/\sum_{u:n[u]=n[t]}\exp\xi_u)$.

In [97]:
importlib.reload(mp5)
#alpha = mp5.todo_fstforward(TLG_sorted)
alpha = { int(k):v for (k,v) in solutions['alpha'].items() }
print(alpha)

{0: 0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 5.616771097666572, 5: 5.616771097666572, 6: 5.616771097666572, 7: 5.616771097666572, 8: 5.616771097666572, 9: 11.233542195333143, 10: 11.233542195333143, 11: 11.233542195333143, 12: 16.157166112439768, 13: 11.233542195333143, 14: 16.157166112439768, 15: 16.850313292999715, 16: 16.157166112439768, 17: 16.850313292999715, 18: 16.157166112439768, 19: 16.850313292999715, 20: 20.38764284898645, 21: 20.38764284898645, 22: 25.311266766093077, 23: 25.311266766093077, 24: 25.311266766093077, 25: 25.311266766093077, 26: 25.311266766093077, 27: 30.92803786375965, 28: 30.92803786375965, 29: 35.85166178086627, 30: 35.85166178086627, 31: 35.85166178086627, 32: 41.468432878532845, 33: 41.468432878532845, 34: 41.468432878532845, 35: 41.468432878532845, 36: 45.1392938271441, 37: 45.1392938271441, 38: 45.1392938271441, 39: 45.1392938271441, 40: 50.75606492481067, 41: 50.75606492481067, 42: 50.75606492481067, 43: 55.27422373380914, 44: 55.27422373380914, 45: 55.274223733

In [99]:
importlib.reload(mp5)
#beta = mp5.todo_fstbackward(TLG_sorted, TLGfinal_sorted)
beta = { int(k):v for (k,v) in solutions['beta'].items() }
print(beta)

{0: 558.5846232397118, 1: 558.5846232397118, 2: 558.5846232397118, 3: 558.5846232397118, 4: 552.9678521420453, 5: 552.9678521420453, 6: 552.9678521420453, 7: 552.9678521420453, 8: 552.9678521420453, 9: 547.3510810443788, 10: 547.3510810443788, 11: 547.3510810443788, 12: 543.1206043078321, 13: 548.0442282249387, 14: 543.1206043078321, 15: 542.4274571272722, 16: 543.1206043078321, 17: 542.4274571272722, 18: 543.1206043078321, 19: 542.4274571272722, 20: 538.1969803907255, 21: 538.1969803907255, 22: 533.2733564736188, 23: 533.2733564736188, 24: 533.2733564736188, 25: 533.2733564736188, 26: 533.2733564736188, 27: 527.6565853759523, 28: 527.6565853759523, 29: 522.7329614588457, 30: 522.7329614588457, 31: 522.7329614588457, 32: 517.1161903611791, 33: 517.1161903611791, 34: 517.1161903611791, 35: 517.1161903611791, 36: 513.4453294125678, 37: 513.4453294125678, 38: 513.4453294125678, 39: 513.4453294125678, 40: 507.8285583149012, 41: 507.8285583149012, 42: 507.8285583149012, 43: 503.310399505902

In [100]:
importlib.reload(mp5)
LG_re,LGfinal_re=mp5.fstreestimate(TLG_sorted, L, Lfinal, alpha, beta)
print(LG_re)

[(0, 'ə', '', 2.723798558070257, 1), (1, '', 'a', 3.1780538303480625, 0), (0, 'ˌei', '', inf, 2), (2, '', 'a', 0.0, 0), (0, 'b', '', 4.158883083359569, 3), (3, 'ˌaʊ', '', inf, 4), (4, 't', '', 0.0, 5), (5, '', 'about', inf, 0), (1, 'b', '', 2.261763098473807, 6), (6, 'ˈaʊ', '', 0.0, 7), (7, 't', '', 0.0, 8), (8, '', 'about', inf, 0), (0, 'ˌɑ', '', inf, 9), (9, 'l', '', 0.0, 10), (10, 'ɛ', '', inf, 11), (11, 'k', '', 0.0, 12), (12, 's', '', 0.0, 13), (13, 'ˈɑ', '', 0.0, 14), (14, 'n', '', 0.0, 15), (15, 'd', '', 0.0, 16), (16, 'ɹ', '', 0.0, 17), (17, 'ɑ', '', inf, 18), (18, 'v', '', 0.0, 19), (19, 'ɪ', '', 0.0, 20), (20, 'tʃ', '', 0.0, 21), (21, '', 'aleksandrovich', inf, 0), (0, 'ˌæ', '', inf, 22), (22, 'l', '', 0.0, 23), (23, 'ɪ', '', 0.0, 24), (24, 'g', '', 0.0, 25), (25, 'z', '', 0.0, 26), (26, 'ˈæ', '', 0.0, 27), (27, 'n', '', 0.0, 28), (28, 'd', '', 0.0, 29), (29, 'ɚ', '', 3.091042453358341, 30), (30, '', 'alexander', inf, 0), (29, 'ə', '', 0.04652001563488284, 31), (31, 'ɹ', '', 

Finally, let's see if this re-estimation fixed the mistake!

In [101]:
importlib.reload(mp5)
TLG_re,TLGfinal_re = mp5.todo_fstcompose(T,Tfinal,LG_re,LGfinal_re)
TLG_sortre,TLGfinal_sortre = mp5.todo_sort_topologically(TLG_re,TLGfinal_re)
delta, psi, bestpath_re = mp5.todo_fstbestpath(TLG_sortre,TLGfinal_sortre)
print([ t[2] for t in bestpath_re if t[2]!='' ])

['whose', 'woods', 'the', 'czar', 'i', 'think', 'i', 'know', 'his', 'house', 'is', 'in', 'the', 'village', 'though', 'he', 'will', 'not', 'see', 'me', 'stopping', 'here', 'to', 'watch', 'his', 'woods', 'fill', 'up', 'with', 'snow', 'me', 'little', 'horse', 'must', 'think', 'it', 'queer', 'to', 'stop', 'without', 'a', 'farm', 'house', 'near', 'between', 'the', 'woods', 'and', 'frozen', 'lake', 'the', 'darkest', 'evening', 'of', 'the', 'year', 'he', 'gives', 'his', 'harness', 'bells', 'a', 'shake', 'to', 'ask', 'if', 'there', 'is', 'some', 'mistake', 'the', 'only', 'other', 'sounds', 'the', 'sweep', 'of', 'easy', 'wind', 'and', 'downy', 'flake', 'the', 'woods', 'are', 'lovely', 'dark', 'and', 'deep', 'but', 'i', 'have', 'promises', 'to', 'keep', 'and', 'miles', 'to', 'go', 'before', 'i', 'sleep', 'and', 'miles', 'to', 'go', 'before', 'i', 'sleep']


Still there!  Maybe it would be a good idea to use language model training data that has some topical similarity to the speech we want to recognize.